# Create models


In [ ]:
import dalex as dx
titanic = dx.datasets.load_titanic()
X = titanic.drop(columns='survived')
y = titanic.survived

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

preprocess = make_column_transformer(
    (StandardScaler(), ['age', 'fare', 'parch', 'sibsp']),
    (OneHotEncoder(), ['gender', 'class', 'embarked']))

## Logistic regression model


In [ ]:
from sklearn.linear_model import LogisticRegression

titanic_lr = make_pipeline(
    preprocess,
    LogisticRegression(penalty = 'l2'))
titanic_lr.fit(X, y)

## Logistic regression model


In [ ]:
from sklearn.ensemble import RandomForestClassifier

titanic_rf = make_pipeline(
    preprocess,
    RandomForestClassifier(max_depth = 3, n_estimators = 500))
titanic_rf.fit(X, y)

## Gradient boosting model


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

titanic_gbc = make_pipeline(
    preprocess,
    GradientBoostingClassifier(n_estimators = 100))
titanic_gbc.fit(X, y)

## Support vector machine model


In [ ]:
from sklearn.svm import SVC

titanic_svm = make_pipeline(
    preprocess,
    SVC(probability = True))
titanic_svm.fit(X, y)

# Models' predictions


In [ ]:
import pandas as pd

johnny_d = pd.DataFrame({'gender': ['male'],
                       'age'     : [8],
                       'class'   : ['1st'],
                       'embarked': ['Southampton'],
                       'fare'    : [72],
                       'sibsp'   : [0],
                       'parch'   : [0]},
                      index = ['JohnnyD'])

henry = pd.DataFrame({'gender'   : ['male'],
                       'age'     : [47],
                       'class'   : ['1st'],
                       'embarked': ['Cherbourg'],
                       'fare'    : [25],
                       'sibsp'   : [0],
                       'parch'   : [0]},
                      index = ['Henry'])

# Instance Level

## Break-down plots for additive attibutions

This concept answer the question: *which variables contribute to this result the most?* 


In [ ]:
import dalex as dx
titanic_rf_exp = dx.Explainer(titanic_rf, X, y, 
                  label = "Titanic RF Pipeline")

In [ ]:
bd_henry = titanic_rf_exp.predict_parts(henry, 
             type = 'break_down')
bd_henry.result

In [ ]:
bd_henry.plot()

In [ ]:
import numpy as np

bd_henry = titanic_rf_exp.predict_parts(henry,
        type = 'break_down',
        order = np.array(['gender', 'class', 'age',
            'embarked', 'fare', 'sibsp', 'parch']))
bd_henry.plot(max_vars = 5)

## Break-down plots for additive interactions

Interaction (deviation from additivity) means that the effect of an explanatory variable depends on the value(s) of other variable(s).


In [ ]:
ibd_henry = titanic_rf_exp.predict_parts(henry, 
                type = 'break_down_interactions', 
                interaction_preference = 10)
ibd_henry.result

In [ ]:
ibd_henry.plot()

## Shapley Additice Explanations (SHAP) for average attributions

To remove the influence of the ordering of the variables.


In [ ]:
bd_henry = titanic_rf_exp.predict_parts(henry, type = 'shap')
bd_henry.result

In [ ]:
bd_henry.plot()

## Local Interpretable Model-agnostic Explanations (LIME)
Break-down (BD) plots and Shapley values, are most suitable for models with a small or moderate number of explanatory variables.
The most popular example of such sparse explainers is the Local Interpretable Model-agnostic Explanations (LIME) method and its modifications.


In the first step, we read the Titanic data and encode categorical variables. In this case, we use the simplest encoding for gender, class, and embarked, i.e., the label-encoding.


In [ ]:
import dalex as dx

titanic = dx.datasets.load_titanic()
X = titanic.drop(columns='survived')
y = titanic.survived

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

X['gender']   = le.fit_transform(X['gender'])
X['class']    = le.fit_transform(X['class'])
X['embarked'] = le.fit_transform(X['embarked'])

In the next step we train a random forest model.


In [ ]:
from sklearn.ensemble import RandomForestClassifier as rfc
titanic_fr = rfc()
titanic_fr.fit(X, y)

It is time to define the observation for which model prediction will be explained. We write Henry’s data into `pandas.Series` object.


In [ ]:
import pandas as pd
henry = pd.Series([1, 47.0, 0, 1, 25.0, 0, 0], 
                  index =['gender', 'age', 'class', 'embarked',
                          'fare', 'sibsp', 'parch']) 

The `lime` library explains models that operate on images, text, or tabular data. In the latter case, we have to use the `LimeTabularExplainer` module.


In [ ]:
from lime.lime_tabular import LimeTabularExplainer 
explainer = LimeTabularExplainer(X, 
                      feature_names=X.columns, 
                      class_names=['died', 'survived'], 
                      discretize_continuous=False, 
                      verbose=True)

The result is an explainer that can be used to interpret a model around specific observations. In the following example, we explain the behaviour of the model for Henry. The `explain_instance()` method finds a local approximation with an interpretable linear model. The result can be presented graphically with the `show_in_notebook()` method.


In [ ]:
lime = explainer.explain_instance(henry, titanic_fr.predict_proba)
lime.show_in_notebook(show_table=True)

## Ceteris-paribus profiles

*"Ceteris paribus"* is a Latin phrase meaning "other things held constant" or "all else unchanged".

Ceteris-paribus (CP) profiles show how a model’s prediction would change if the value of a single exploratory variable changed. 


In [ ]:
import dalex as dx
titanic = dx.datasets.load_titanic()
X = titanic.drop(columns='survived')
y = titanic.survived

In [ ]:
from sklearn.ensemble import RandomForestClassifier

titanic_rf = make_pipeline(
    preprocess,
    RandomForestClassifier(max_depth = 3, n_estimators = 500))
titanic_rf.fit(X, y)

In [ ]:
henry = pd.DataFrame({'gender'   : ['male'],
                       'age'     : [47],
                       'class'   : ['1st'],
                       'embarked': ['Cherbourg'],
                       'fare'    : [25],
                       'sibsp'   : [0],
                       'parch'   : [0]},
                      index = ['Henry'])


import dalex as dx
titanic_rf_exp = dx.Explainer(titanic_rf, X, y, 
                  label = "Titanic RF Pipeline")

In [ ]:
cp_henry = titanic_rf_exp.predict_profile(henry)
cp_henry.result

In [ ]:
cp_henry.plot(variables = ['age', 'fare'])

In [ ]:
cp_henry.plot(variables = ['class', 'embarked'],
               variable_type = 'categorical')

In [ ]:
from sklearn.linear_model import LogisticRegression

titanic_lr = make_pipeline(
    preprocess,
    LogisticRegression(penalty = 'l2'))
titanic_lr.fit(X, y)

In [ ]:
import dalex as dx
titanic_lr_exp = dx.Explainer(titanic_lr, X, y, 
                  label = "Titanic RL Pipeline")

In [ ]:
cp_henry2 = titanic_lr_exp.predict_profile(henry)
cp_henry.plot(cp_henry2, variables = ['age', 'fare'])

## Ceteris-paribus oscillations





# Dataset Level
